# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7efe6cf7d9a0>
├── 'a' --> tensor([[-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765]])
└── 'x' --> <FastTreeValue 0x7efe810ba760>
    └── 'c' --> tensor([[ 1.5277, -0.8325,  0.1780,  1.0732],
                        [-0.1768,  1.0798,  0.2224,  0.1723],
                        [ 0.0393, -1.0571, -1.4041,  0.4009]])

In [4]:
t.a

tensor([[-0.2425,  1.4784,  0.1745],
        [ 0.9078,  0.0636,  0.9765]])

In [5]:
%timeit t.a

72.1 ns ± 0.104 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7efe6cf7d9a0>
├── 'a' --> tensor([[ 1.1347, -2.5695,  1.7699],
│                   [-0.7178,  0.3784, -0.5087]])
└── 'x' --> <FastTreeValue 0x7efe810ba760>
    └── 'c' --> tensor([[ 1.5277, -0.8325,  0.1780,  1.0732],
                        [-0.1768,  1.0798,  0.2224,  0.1723],
                        [ 0.0393, -1.0571, -1.4041,  0.4009]])

In [7]:
%timeit t.a = new_value

73.7 ns ± 0.052 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765]]),
    x: Batch(
           c: tensor([[ 1.5277, -0.8325,  0.1780,  1.0732],
                      [-0.1768,  1.0798,  0.2224,  0.1723],
                      [ 0.0393, -1.0571, -1.4041,  0.4009]]),
       ),
)

In [10]:
b.a

tensor([[-0.2425,  1.4784,  0.1745],
        [ 0.9078,  0.0636,  0.9765]])

In [11]:
%timeit b.a

69.9 ns ± 0.0582 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.0994, -0.4507,  0.5695],
               [ 0.4978,  0.9433, -2.4549]]),
    x: Batch(
           c: tensor([[ 1.5277, -0.8325,  0.1780,  1.0732],
                      [-0.1768,  1.0798,  0.2224,  0.1723],
                      [ 0.0393, -1.0571, -1.4041,  0.4009]]),
       ),
)

In [13]:
%timeit b.a = new_value

549 ns ± 0.0874 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

918 ns ± 6.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 42.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

146 µs ± 604 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 928 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7efdb95c40d0>
├── 'a' --> tensor([[[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]],
│           
│                   [[-0.2425,  1.4784,  0.1745],
│                    [ 0.9078,  0.0636,  0.9765]]])
└── 'x' --> <FastTreeValue 0x7efdb93c5670>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7efe810d2a90>
├── 'a' --> tensor([[-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765],
│                   [-0.2425,  1.4784,  0.1745],
│                   [ 0.9078,  0.0636,  0.9765]])
└── 'x' --> <FastTreeValue 0x7efdb1213df0>
    └── 'c' --> tensor([[ 1.5277, -0.8325,  0.1780,  1.0732],
                        [-0.1768,  1.0798,  0.2224,  0.1723],
                 

In [23]:
%timeit t_cat(trees)

31.5 µs ± 58 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.2 µs ± 68.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]],
       
               [[-0.2425,  1.4784,  0.1745],
                [ 0.9078,  0.0636,  0.9765]]]),
    x: Batch(
           c: tensor([[[ 1.5277, -0.8325,  0.1780,  1.0732],
                       [-0.1768,  1.0798,  0.2224,  0.1723],
                       [ 0.0393, -1.0571, -1.4041,  0.4009]],
         

In [26]:
%timeit Batch.stack(batches)

83.2 µs ± 163 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765],
               [-0.2425,  1.4784,  0.1745],
               [ 0.9078,  0.0636,  0.9765]]),
    x: Batch(
           c: tensor([[ 1.5277, -0.8325,  0.1780,  1.0732],
                      [-0.1768,  1.0798,  0.2224,  0.1723],
                      [ 0.0393, -1.0571, -1.4041,  0.4009],
                      [ 1.5277, -0.8325,  0.1780,  1.0732],
                      [-0.1768,  

In [28]:
%timeit Batch.cat(batches)

153 µs ± 520 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

924 µs ± 336 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
